# Import packages

In [ ]:
%load_ext autoreload
%autoreload 2

import os, sys, sys
from pathlib import Path
for p in [Path.cwd()] + list(Path.cwd().parents):
    if p.name == 'Multifirefly-Project':
        os.chdir(p)
        sys.path.insert(0, str(p / 'multiff_analysis/multiff_code/methods'))
        break

from data_wrangling import specific_utils, process_monkey_information, general_utils
from pattern_discovery import pattern_by_trials, pattern_by_trials, cluster_analysis, organize_patterns_and_features
from visualization.matplotlib_tools import plot_behaviors_utils
from neural_data_analysis.neural_analysis_tools.get_neural_data import neural_data_processing
from neural_data_analysis.neural_analysis_tools.visualize_neural_data import plot_neural_data, plot_modeling_result
from neural_data_analysis.neural_analysis_tools.model_neural_data import transform_vars, neural_data_modeling, drop_high_corr_vars, drop_high_vif_vars
from neural_data_analysis.topic_based_neural_analysis.neural_vs_behavioral import prep_monkey_data, prep_target_data, neural_vs_behavioral_class
from neural_data_analysis.topic_based_neural_analysis.planning_and_neural import planning_and_neural_class, pn_utils, pn_helper_class
from neural_data_analysis.neural_analysis_tools.cca_methods import cca_class
from neural_data_analysis.neural_analysis_tools.cca_methods import cca_class, cca_utils, cca_cv_utils
from neural_data_analysis.neural_analysis_tools.cca_methods.cca_plotting import cca_plotting, cca_plot_lag_vs_no_lag, cca_plot_cv
from machine_learning.ml_methods import regression_utils, ml_methods_utils, regz_regression_utils, ml_methods_class, classification_utils, ml_plotting_utils
from neural_data_analysis.design_kits.design_by_segment import create_design_df, predictor_utils, other_feats
from neural_data_analysis.topic_based_neural_analysis.planning_and_neural import planning_and_neural_class, pn_utils, pn_helper_class, pn_aligned_by_seg, pn_aligned_by_event, pn_glm_utils
from neural_data_analysis.neural_analysis_tools.glm_tools.tpg import glm_bases, glm_plotting, glm_plotting2, glm_fit

from neural_data_analysis.topic_based_neural_analysis.stop_event_analysis.stop_psth import core_stops_psth, psth_postprocessing, psth_stats, compare_events, dpca_utils
from neural_data_analysis.topic_based_neural_analysis.stop_event_analysis.stop_glm.glm_fit import stop_glm_fit, cv_stop_glm, glm_fit_utils, variance_explained
from neural_data_analysis.topic_based_neural_analysis.stop_event_analysis.stop_glm.glm_plotting import plot_spikes, plot_glm_fit, plot_tuning_func
from neural_data_analysis.design_kits.design_around_event import event_binning, stop_design, cluster_design, design_checks
from neural_data_analysis.topic_based_neural_analysis.stop_event_analysis.stop_glm.glm_hyperparams import compare_glm_configs, glm_hyperparams_class
from neural_data_analysis.design_kits.design_by_segment import spike_history

import sys
import math
import gc
import subprocess
from pathlib import Path
from importlib import reload

# Third-party imports
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import rc
from scipy import linalg, interpolate
from scipy.signal import fftconvolve
from scipy.io import loadmat
from scipy import sparse
from numpy import pi

from sklearn.metrics import roc_auc_score, average_precision_score, roc_curve, precision_recall_curve
# Machine Learning imports
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.multivariate.cancorr import CanCorr

# Neuroscience specific imports
import neo
import rcca

plt.rcParams["animation.html"] = "html5"
os.environ['KMP_DUPLICATE_LIB_OK']='True'
rc('animation', html='jshtml')
matplotlib.rcParams.update(matplotlib.rcParamsDefault)
matplotlib.rcParams['animation.embed_limit'] = 2**128
pd.set_option('display.float_format', lambda x: '%.5f' % x)
np.set_printoptions(suppress=True)
print("done")

%load_ext autoreload
%autoreload 2

# Retrieve data

## get data

In [ ]:
raw_data_folder_path = "all_monkey_data/raw_monkey_data/monkey_Schro/data_0416"

In [ ]:
raw_data_folder_path = "all_monkey_data/raw_monkey_data/monkey_Bruno/data_0330"
raw_data_folder_path = "all_monkey_data/raw_monkey_data/monkey_Bruno/data_0312"

In [ ]:
raw_data_folder_path = "all_monkey_data/raw_monkey_data/monkey_Bruno/data_0330"

In [ ]:
reload(pn_aligned_by_event)
reload(pn_utils)

In [ ]:
reduce_y_var_lags = False
planning_data_by_point_exists_ok = True
y_data_exists_ok = True

pn = pn_aligned_by_event.PlanningAndNeuralEventAligned(raw_data_folder_path=raw_data_folder_path)
pn.prep_data_to_analyze_planning(planning_data_by_point_exists_ok=planning_data_by_point_exists_ok)
#pn.get_x_and_y_data_for_modeling(exists_ok=y_data_exists_ok, reduce_y_var_lags=reduce_y_var_lags)

pn.rebin_data_in_new_segments(cur_or_nxt='cur', first_or_last='first', time_limit_to_count_sighting=2,
                                 start_t_rel_event=0, end_t_rel_event=1.5, rebinned_max_x_lag_number=2)

for col in ['cur_vis', 'nxt_vis', 'cur_in_memory', 'nxt_in_memory']:
    pn.rebinned_y_var[col] = (pn.rebinned_y_var[col] > 0).astype(int)
    
rebinned_x_var = pn.rebinned_x_var.copy()
rebinned_y_var = pn.rebinned_y_var.copy()

rebinned_y_var.shape


## select subset

### get heading data

In [ ]:
heading_info_df, heading_df = pn_glm_utils.get_test_heading_df(raw_data_folder_path)

In [ ]:
# rebinned_x_var, rebinned_y_var = pn_glm_utils.select_ff_subset(heading_df, pn.rebinned_x_var, pn.rebinned_y_var, 
#                                                                top=False, pct=0.5)
# rebinned_x_var = pn_glm_utils.drop_constant_columns(rebinned_x_var)

# GLM data

In [ ]:
reload(create_design_df)
reload(other_feats)

data = rebinned_y_var.copy()
trial_ids = data['new_segment']
dt = pn.bin_width

design_df, meta0, meta = create_design_df.get_initial_design_df(data, dt, trial_ids)

cluster_num = 3
y = rebinned_x_var[f'cluster_{cluster_num}']

# design_df, meta = temporal_feats.add_spike_history(
#     design_df, y, meta0['trial_ids'], dt,
#     n_basis=4, t_max=0.20, edge='zero',
#     prefix='spk_hist', style='bjk',
#     meta=meta
# )

chk = predictor_utils.check_design_vs_bases(design_df, meta, strict=True)
assert chk['ok'], chk['problems']


cluster_cols = [col for col in rebinned_x_var.columns if col.startswith('cluster_')]
df_Y = rebinned_x_var[cluster_cols]
df_Y.columns = df_Y.columns.str.replace('cluster_', '').astype(int)



In [ ]:
meta

In [ ]:
design_df.columns

## other ways of taking out data

In [ ]:
# #df_X = design_df

# # cur_ff_distance_cols = [col for col in design_df.columns if 'cur_ff_distance' in col]
# # df_X = design_df[cur_ff_distance_cols + ['const']]

cur_ff_cols = [col for col in design_df.columns if ('cur_' in col) and ('cur_in_memory' not in col)]
nxt_ff_cols = [col for col in design_df.columns if 'nxt_' in col]

# # df_X = design_selected

# # df_X = design_selected.copy()

# df_X = design_df[['speed_z', 'time_since_last_capture',
#        'ang_accel_mag_spline:s0', 'ang_accel_mag_spline:s1',
#        'ang_accel_mag_spline:s2', 'ang_accel_mag_spline:s3']].copy()
# df_X[cur_ff_distance_cols] = design_df[cur_ff_distance_cols].copy()

# print('df_X.shape:', df_X.shape)


In [ ]:
use_this = False
if use_this:
    df_X = design_df[['speed_z', 'time_since_last_capture',
        'ang_accel_mag_spline:s0', 'ang_accel_mag_spline:s1',
        'ang_accel_mag_spline:s2', 'ang_accel_mag_spline:s3'] + cur_ff_cols + nxt_ff_cols].copy()

    print('df_X.shape:', df_X.shape)
else:
    df_X = design_df.copy()

df_X['random_0_or_1'] = np.random.randint(0, 2, len(df_X))

In [ ]:
stop!

# Fit glm

In [ ]:
X_pruned, vif_report = design_checks.check_design(df_X)

In [ ]:
# X_pruned_new, vif_report = design_checks.check_design(X_pruned)

In [ ]:
exposure = np.full_like(y, fill_value=pn.bin_width, dtype=float)
offset_log = np.log(exposure)

report = stop_glm_fit.glm_mini_report(
    # df_X=df_X, 
    df_X=X_pruned,
    df_Y=df_Y, offset_log=offset_log,
    cov_type='HC1', 
    fast_mle=True,
    do_inference=False, 
    make_plots=True,
    show_plots=True,
)

In [ ]:
stop!

In [ ]:
report['coefs_df']

In [ ]:
report['coefs_df'][report['coefs_df']['term'] == 'random_0_or_1']

In [ ]:
report

# Add spike history

In [ ]:
dt = 0.02
t_max = 0.20

# 1) Rebin with padding
design_pad, pad_info = spike_history.rebin_spike_data_with_pad(
    spikes_df=pn.spikes_df,
    new_seg_info=pn.new_seg_info,
    bin_width=dt,
    t_max=t_max,
)

# 2) Precompute history for all neurons
X_hist, basis, colnames = spike_history.compute_spike_history_designs(
    design_pad,
    rebinned_x_var,
    n_pad_bins=pad_info['n_pad_bins'],
    dt=dt,
    t_max=t_max,
    n_basis=5,
)

# 3) For now, since we include both self-history and cross-history, the design_df is the same for all neurons
spike_cols = [c for c in design_pad.columns if c.startswith('cluster_')]

for target_col in spike_cols:
    cross_neurons = [c for c in spike_cols if c != target_col]

    design_w_history, meta['groups'] = spike_history.add_spike_history_to_design(
        X_pruned,
        colnames,
        X_hist,
        target_col,
        include_self=True,
        cross_neurons=cross_neurons,
        meta_groups=meta['groups'],
    )
    break

In [ ]:
X_pruned1, vif_report = design_checks.check_design(design_w_history)
print('Selected final columns: ', X_pruned1.columns)

In [ ]:
# X_pruned2, vif_report = design_checks.check_design(X_pruned1)

In [ ]:
do_inference = True

exposure = np.full_like(y, fill_value=pn.bin_width, dtype=float)
offset_log = np.log(exposure)

report = stop_glm_fit.glm_mini_report(
    df_X=X_pruned1,
    df_Y=df_Y, offset_log=offset_log,
    cov_type='HC1', 
    fast_mle=True,
    do_inference=do_inference, 
    make_plots=True,
    show_plots=True,
    meta_groups=meta['groups']
)

In [ ]:
coefs_df = report['coefs_df']
coefs_df

In [ ]:
general_utils.check_na_in_df(report['coefs_df'], df_name="DataFrame", return_rows_and_columns=False)

In [ ]:
stop!

# GLM analysis

## check fitting


In [ ]:
coefs_df    = report['coefs_df']
metrics_df  = report['metrics_df']
pop_tests   = report['population_tests_df']
metrics_df

## VE

In [ ]:
results = variance_explained.run_variance_explained_analysis(
    report=report,
    df_X=df_X,
    df_Y=df_Y,
    offset_log=offset_log,
    event_ids=rebinned_x_var['new_segment'].to_numpy(),
    k=10,
)

print('=' * 80)
print(pd.Series(results['summary_metrics']))
print(results['per_event_df'].head())


## population latent

In [ ]:
from neural_data_analysis.neural_analysis_tools.glm_tools.glm_eval import shared_manifold, parity_utils, population_latent_benchmark

In [ ]:
time_idx = data['new_bin']
condition_labels = None

res = population_latent_benchmark.population_latent_benchmark_function(
    X, X_hat, trial_ids, time_idx, condition_labels,
    fa_factors=8,
    max_rank=10,
    shuffle_mode='latent'
)

res['figs']['cca_spectrum']
res['figs']['ve_summary']
res['figs']['rrr_curve']


In [ ]:
from neural_data_analysis.neural_analysis_tools.glm_tools.glm_eval.shared_manifold import plot_shared_components
import pandas as pd

df_lat = pd.concat(
    res['parity']['latents_by_fold'].values(),
    ignore_index=True
)

best_rank = res['rrr']['best_rank']

figs = plot_shared_components(
    df_lat,
    components=[1, 2], # or use n_components=best_rank instead
    overlay_mode='separate'
)


In [ ]:
# Option 1: one fold
fig = shared_manifold.plot_shared_phase_plane(
    res['parity']['latents_by_fold'][0],
    components=(1, 2),
    smooth_bins=3
)

# or concat all folds (one caveat: Different folds use slightly different CCA fits.)

df_lat = pd.concat(
    res['parity']['latents_by_fold'].values(),
    ignore_index=True
)

fig = shared_manifold.plot_shared_phase_plane(
    df_lat,
    components=(1, 2),
    smooth_bins=3
)


fig = shared_manifold.plot_shared_phase_plane_by_fold(
    res['parity']['latents_by_fold'],
    components=(1, 2),
    smooth_bins=3,
    alpha_fold=0.6
)


## Plot spikes

In [ ]:
cluster_idx = 4

rebinned_y_var['rel_time'] = rebinned_y_var['bin_mid_time_rel_to_event']

for event_id in range(40, 52):
    # If your GLM used offset_log = np.log(exposure_s), you can omit exposure_s:
    plot_spikes.plot_observed_vs_predicted_event(
        #binned_feats_sc=df_X,
        binned_feats_sc=X_pruned1,
        binned_spikes=df_Y,
        meta_used=rebinned_y_var,
        offset_log=offset_log,
        model_res=report['results'][cluster_idx],   # GLM for cluster 0
        cluster_idx=cluster_idx,
        seg_id=event_id,
        time_col='rel_time',
        seg_col='new_segment'
    )
plt.show()

## tuning curve

In [ ]:
for cluster_id in df_Y.columns:
    
    print('==========================================')
    model_res = report['results'][cluster_id]
    
    # for var in ['nxt_in_memory', 'cur_ff_distance', 'log_cur_ff_distance']:
    for var in ['cur_vis_off:b0:0', 'ang_accel_mag_spline:s2']:

        exposure_s = np.exp(offset_log)
        # make empirical curve
        tc_emp = plot_tuning_func.empirical_tuning_curve(
            binned_spikes=df_Y[cluster_id].to_numpy(),
            predictor_vals=df_X[var].to_numpy(),
            exposure_s=exposure_s,
            nbins=20
        )

        # # just empirical
        # plot_tuning_func.plot_tuning_with_ci(tc_emp, title='Distance → rate (empirical)',
        #             kind='line', ci_style='band', show_counts=True)
        # plt.show()


        # make GLM curve
        tc_glm = plot_tuning_func.glm_tuning_curve(
            model_res, X_pruned1,
            var=var,
            offset_log=offset_log,
            average='marginal',
            weights=exposure_s,                 # time-weighted average rate (recommended)
            return_ci=True
        )

        # overlay
        plot_tuning_func.overlay_tuning_curves(tc_emp, tc_glm, xcol=var,
                            title=f'Unit {cluster_id}: {var} tuning')

In [ ]:
tc_emp = plot_tuning_func.empirical_tuning_curve(binned_spikes=df_Y[cluster_id].to_numpy(),
            predictor_vals=df_X[var].to_numpy(),
            exposure_s=exposure_s,
            nbins=20)
plot_tuning_func.plot_tuning_with_ci(tc_emp, title='Distance → rate (empirical)',
                    kind='line', ci_style='band', show_counts=True)

# # for discrete variables
# df_emp = empirical_tuning_curve(spikes, cur_visible.astype(int), bin_durations)
# plot_tuning_with_ci(df_emp, title='Visibility → rate (empirical)',
#                     kind='bar', ci_style='errorbar', show_counts=True)

# Decoding

In [ ]:
from neural_data_analysis.neural_analysis_tools.decoding_tools.general_decoding import standard_decoding

cols_to_decode = ['cur_vis', 'nxt_vis', 'random_0_or_1']
groups = np.array(data['new_segment'])
standard_decoding.population_decoding_cv(
    cols_to_decode=cols_to_decode,
    df_X=df_X,
    df_Y=df_Y,
    groups=groups,
    decoders=('logreg', 'lda'),
    n_splits=5
)


# try forward selection

In [ ]:
list(meta['groups'].keys())

In [ ]:
meta['groups']

In [ ]:
design_df.columns

In [ ]:
from neural_data_analysis.neural_analysis_tools.glm_tools import forward_selection

In [ ]:
reload(forward_selection)

In [ ]:
design_df

In [ ]:
groups = meta['groups']
df_X_full = design_df

# df_X_full.columns is your long Index([...]) from the message
# groups is exactly your dict
# y is spikes, trial_ids from data, offset_log = np.log(exposure_s)

# sensible tiny base: intercept + one robust driver
base_groups = ['log_cur_ff_distance']  # 'const' is already a column in df_X_full; keep it inside a group if you want
if 'const' not in groups:
    # treat the literal 'const' column as its own group
    groups = {'const': ['const'], **groups}
base_groups = ['const', 'log_cur_ff_distance']

# candidates = everything else (you can curate)
candidates = [g for g in groups.keys() if g not in base_groups]

X_star, selected_groups, log_df = forward_selection.forward_block_select(
    df_X_full=df_X_full,
    groups=groups,
    y=y,
    trial_ids=data['new_segment'].to_numpy(),
    offset_log=offset_log,  # or scalar np.log(dt)
    base_groups=base_groups,
    candidate_groups=candidates,
    nfolds=5,
    max_steps=20,
    rng=0
)

print('Selected order:', selected_groups)
print('Final X shape:', X_star.shape)
print(log_df)

In [ ]:
log_df['added_group'].values

In [ ]:
# groups: dict[str, list[str]]
# design_df: your full design matrix
# log_df: from forward selection

# which groups were actually added (drop the None base row)
selected_groups = [g for g in log_df['added_group'].dropna().unique() if g in groups]

# flatten the column list for these groups
selected_cols = sum((groups[g] for g in selected_groups), [])

# keep only those columns
design_selected = design_df[selected_cols].copy()

print('Groups:', selected_groups)
print('Cols:', selected_cols[:10], '...')
print(design_selected.shape)

In [ ]:
## What groups almost made it?

# # quick “one-more-block” screen
# base = ['const','log_cur_ff_distance','LDz_mag']
# remaining = [g for g in groups if g not in base]
# base_X = df_X_full[sum((groups.get(g, []) for g in base), [])].copy()

# def cv_dev(X):
#     return forward_selection._cv_dev_poisson(
#         y, X, trial_ids=data['new_segment'].to_numpy(),
#         offset_log=offset_log, nfolds=5, rng=0
#     )

# base_score = cv_dev(base_X)
# rows = []
# for g in remaining:
#     cols = [c for c in groups[g] if c in df_X_full.columns]
#     if not cols: 
#         continue
#     X_try = base_X.join(df_X_full[cols])
#     s = cv_dev(X_try)
#     rows.append((g, s, (base_score - s)/max(base_score, 1e-12)))
# rank = (pd.DataFrame(rows, columns=['group','cv_dev','rel_improve'])
#           .sort_values('cv_dev').head(10))
# print(rank)

# fit single neuron (w spike history)

## glm data

In [ ]:
from neural_data_analysis.design_kits.design_by_segment import temporal_feats

In [ ]:
data = rebinned_y_var.copy()
trial_ids = data['new_segment']
dt = pn.bin_width

design_df, meta0, meta = create_design_df.get_initial_design_df(data, dt, trial_ids)

cluster_num = 3
y = rebinned_x_var[f'cluster_{cluster_num}']

design_df, meta = temporal_feats.add_spike_history(
    design_df, y, meta0['trial_ids'], dt,
    n_basis=4, t_max=0.20, edge='zero',
    prefix='spk_hist', style='bjk',
    meta=meta
)

chk = predictor_utils.check_design_vs_bases(design_df, meta, strict=True)
assert chk['ok'], chk['problems']

## fit

In [ ]:
res = glm_fit.fit_poisson_glm_trials(
    design_df, y,
    dt=dt,
    trial_ids=meta0['trial_ids'],
    add_const=False,            # we already inserted 'const' in design_df
    cluster_se=True
)

## check columns

In [ ]:
df2, info = predictor_utils.drop_aliased_columns(design_df)
info['dropped']

In [ ]:
# 1) rank of the history block
H = design_df.filter(regex=r'^spk_hist:').to_numpy()
print("hist shape:", H.shape, "rank:", np.linalg.matrix_rank(H))

# 2) condition / singular values
sv = np.linalg.svd(H, compute_uv=False)
print("singular values:", sv)

# 3) is that column almost in the span of the others?
import numpy as np
c = design_df['spk_hist:b0:3'].to_numpy()
R = design_df[[c for c in design_df.columns if c.startswith('spk_hist:') and c != 'spk_hist:b0:3']].to_numpy()
beta, *_ = np.linalg.lstsq(R, c, rcond=None)
resid = c - R @ beta
print("relative residual:", np.linalg.norm(resid)/max(np.linalg.norm(c), 1e-12))

## check fr

In [ ]:
dt = 0.04
rates = df_Y.sum(axis=0) / (len(df_Y) * dt)
# rates is a Series indexed by unit, in Hz
rates

In [ ]:
import pandas as pd

def firing_rates_from_df(spikes_df, time_col='time', cluster_col='cluster'):
    start_s = spikes_df[time_col].min()
    end_s = spikes_df[time_col].max()
    duration = end_s - start_s

    counts = spikes_df.groupby(cluster_col).size()
    rates_hz = counts / duration
    return rates_hz.rename('rate_hz').reset_index()

firing_rates_from_df(pn.spikes_df)

## plot

In [ ]:
theta, f, std, info =glm_plotting.plot_angle_tuning_function(
    res, design_df, meta,
    base_prefix='cur_ff_angle',
    M=None,                 # auto-detect harmonics
    polar=False,             # pretty polar plot
    z=1.96                  # 95% CI
)

theta, f, std, info =glm_plotting.plot_angle_tuning_function(
    res, design_df, meta,
    base_prefix='cur_ff_angle',
    M=None,                 # auto-detect harmonics
    polar=True,             # pretty polar plot
    z=1.96                  # 95% CI
)

In [ ]:
glm_plotting.plot_fitted_kernels(
    res, design_df, meta, dt,
    prefixes=['cur_vis_on','cur_vis_off','nxt_vis_on','nxt_vis_off','spk_hist'],
    z=1.96  # 95% CI
)